**Programming Basic Concepts:**
1. Algorithm
2. Source code
3. Compiler
4. Data Type
5. Variable
6. Conditionals
7. Array
8. Loop
9. Functions
10. Class



Importing Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time


In [ ]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim


Here we are creating empty lists to store the values

In [ ]:
url = []
property_label=[]
property_price = []
description=[]
property_location = []
date_listed = []


Now we would read the number of pages

In [ ]:
for page in range(1,5):
    html_string = requests.get("https://www.gumtree.com.au/s-property-for-rent/melbourne/page-"+str(page)+"/c18364l3001317").content
    soup = BeautifulSoup(html_string, 'html.parser')
    
    
    content = soup.find_all("div",attrs={"class": "user-ad-collection-new-design"})
    for ad_list in content:
        for ad in ad_list.find_all('a'):
            ad_url = ad['href']
            url.append(ad_url)
            ad_basics = ad['aria-label'].split('\n')
            ad_title , price , location_date = ad_basics[0],ad_basics[1].strip(),ad_basics[2].strip()
            cost = re.findall('Price:([^.]+)',price)[0]       #extracting price
            property_price.append(cost)
            property_label.append(ad_title)
            loc_date=location_date.split('.')
            loc,date= loc_date[0],loc_date[1]        
            location= re.findall('Location:([^.]+)',loc)[0]     #extracting location 
            property_location.append(location)
            desc= ad.find('p',class_='user-ad-row-new-design__description-text').text
            description.append(desc)


Then we have to create a dataframe.

In [ ]:
column = {'Location' :property_location, 'Title':property_label,'Description' : description, 'Price' : property_price, 'Link':url}
result = pd.DataFrame(column)
result.head(5)

,Location,Title,Description,Price,Link
0,"Whittlesea Area, Lalor",3 Bedroom House for Lease - 1/19 Canberra Gro...,Family home in the perfect location! \nLocated...,$400,/s-ad/lalor/property-for-rent/3-bedroom-house-...
1,"Melbourne City, Melbourne CBD",Immaculate CBD 2 bedroom 2 bathroom Apartment.,Apartment 1906/33 Mackenzie Street Melbourne \...,$780,/s-ad/melbourne-cbd/property-for-rent/immacula...
2,"Melbourne City, Melbourne CBD",Stunning Drewery Lane CBD Apartment.,Apartment 35/7 Drewery Lane Melbourne \n***SPE...,$780,/s-ad/melbourne-cbd/property-for-rent/stunning...
3,"Frankston Area, Carrum Downs",House for rent with BIG yard space.,"House in Carrums down , Victoria\n642 Franksto...",$490,/s-ad/carrum-downs/property-for-rent/house-for...
4,"Greater Dandenong, Noble Park North",Studio room for rent .,Studio room available for rent for $260 includ...,$270,/s-ad/noble-park-north/property-for-rent/studi...


Finding latitude and longitude of the property location using geopy.

In [ ]:
geolocator = Nominatim(user_agent='New_application')
for index, row in result.iterrows():
    location = geolocator.geocode(row['Location'],timeout=10)
    if location == None:
        result.loc[index,'latitude'] = 'Not available'
        result.loc[index,'longitude'] = 'Not available'
    else:   
        lat,lng = location.latitude , location.longitude
        result.loc[index,'latitude'] = lat
        result.loc[index,'longitude'] = lng  

In [ ]:
result

,Location,Title,Description,Price,Link,latitude,longitude
0,"Whittlesea Area, Lalor",3 Bedroom House for Lease - 1/19 Canberra Gro...,Family home in the perfect location! \nLocated...,$400,/s-ad/lalor/property-for-rent/3-bedroom-house-...,Not available,Not available
1,"Melbourne City, Melbourne CBD",Immaculate CBD 2 bedroom 2 bathroom Apartment.,Apartment 1906/33 Mackenzie Street Melbourne \...,$780,/s-ad/melbourne-cbd/property-for-rent/immacula...,-37.815283,144.961879
2,"Melbourne City, Melbourne CBD",Stunning Drewery Lane CBD Apartment.,Apartment 35/7 Drewery Lane Melbourne \n***SPE...,$780,/s-ad/melbourne-cbd/property-for-rent/stunning...,-37.815283,144.961879
3,"Frankston Area, Carrum Downs",House for rent with BIG yard space.,"House in Carrums down , Victoria\n642 Franksto...",$490,/s-ad/carrum-downs/property-for-rent/house-for...,Not available,Not available
4,"Greater Dandenong, Noble Park North",Studio room for rent .,Studio room available for rent for $260 includ...,$270,/s-ad/noble-park-north/property-for-rent/studi...,-37.95127,145.190838
...,...,...,...,...,...,...,...
115,"Monash Area, Mount Waverley",Mount Waverley 2 bedroom house rental lease.,- Mount Waverly\n- $470 per week\n- 2 bedroom\...,$480,/s-ad/mount-waverley/property-for-rent/mount-w...,Not available,Not available
116,"Melton Area, Caroline Springs",Wanted: Furnished Bedroom Vacant!!.,"Nice clean house of 5 bedroom, looking for tru...",not listed,/s-ad/caroline-springs/property-for-rent/furni...,Not available,Not available
117,"Melton Area, Melton South",Room for rent.,"A decent size room with a double bed, built in...",$500,/s-ad/melton-south/property-for-rent/room-for-...,-32.341434,139.398844
118,"Wyndham Area, Werribee",Married couple looking for a 4 bedroom house t...,Hello my husband and myself with our 2 childre...,$300 negotiable,/s-ad/werribee/property-for-rent/married-coupl...,Not available,Not available


Plotting the map of melbourne using folium

In [ ]:

import folium
mel_map = folium.Map([-37.840935, 144.946457], zoom_start=10)
mel_map

Plotting the details of property 

In [ ]:

for index, row in result.iterrows():
    if row['latitude'] != 'Not available':
        link_for_property = "https://www.gumtree.com.au"+row['Link']
        folium.Marker([row['latitude'], row['longitude']],
                      popup=row['Price']+"\n"+'<a href='+link_for_property+' target="_blank">Property Link</a>',
                      tooltip=row['Location'],              
                     ).add_to(mel_map)        
mel_map